In [2]:
!pip install feast scikit-learn pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8

In [3]:
!feast init feature_repo



Creating a new Feast repository in /content/feature_repo.



creates a directory named feature_repo containing:​

feature_store.yaml: Configuration file for your feature store.​

Example Feature Definitions: Sample Python files illustrating feature definitions.

In [5]:
%cd feature_repo


/content/feature_repo/feature_repo


In [6]:
!feast apply

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_len" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/content/feature_repo/feature_repo/example_repo.py:27: DeprecationWarning: Entity value_type will be mandatory in the next release. Please specify a value_type for entity 'driver'.
  driver = Entity(name="driver", join_keys=["driver_id"])
Applying changes for project feature_repo
/usr/local/lib/python3.11/dist-packages/feast/feature_store.py:581: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(
No changes to registry
No changes to infrastructure


In [17]:
#Load and Explore the Data
import pandas as pd

df = pd.read_parquet('/content/feature_repo/feature_repo/data/driver_stats.parquet')

df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2025-03-24 10:00:00+00:00,1005,0.655618,0.291158,806,2025-04-08 10:24:01.908
1,2025-03-24 11:00:00+00:00,1005,0.849991,0.882946,750,2025-04-08 10:24:01.908
2,2025-03-24 12:00:00+00:00,1005,0.603516,0.368824,946,2025-04-08 10:24:01.908
3,2025-03-24 13:00:00+00:00,1005,0.727313,0.559777,500,2025-04-08 10:24:01.908
4,2025-03-24 14:00:00+00:00,1005,0.578790,0.694617,537,2025-04-08 10:24:01.908


In [18]:
len(df)

1807

Generate a Training Dataset from Feast

In [19]:
from feast import FeatureStore
from datetime import datetime

store = FeatureStore(repo_path=".")

# Define entity dataframe
entity_df = df[['driver_id', 'event_timestamp']].drop_duplicates().sample(n=150, random_state=42).copy()

entity_df = entity_df.reset_index(drop=True)


# Retrieve historical features from Feast
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips"
    ],
).to_df()

# Show the training data
training_df.head()


,driver_id,event_timestamp,conv_rate,acc_rate,avg_daily_trips
0,1002,2025-03-24 11:00:00+00:00,0.059025,0.477846,984
1,1001,2025-03-24 13:00:00+00:00,0.182314,0.836814,465
2,1001,2025-03-24 18:00:00+00:00,0.622156,0.589965,555
3,1003,2025-03-25 00:00:00+00:00,0.143273,0.180927,638
4,1003,2025-03-25 08:00:00+00:00,0.390656,0.892241,809


Let's create a synthetic label to simulate an ML use case (e.g., predicting driver performance):

In [20]:
import numpy as np

# Add a synthetic binary target variable for demonstration
np.random.seed(42)
training_df['high_performance'] = np.random.choice([0, 1], size=len(training_df))

# Features and labels
X = training_df[["conv_rate", "acc_rate", "avg_daily_trips"]]
y = training_df["high_performance"]


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.43333333333333335
              precision    recall  f1-score   support

           0       0.35      0.50      0.41        12
           1       0.54      0.39      0.45        18

    accuracy                           0.43        30
   macro avg       0.45      0.44      0.43        30
weighted avg       0.46      0.43      0.44        30



Materialize to Online Store

In [22]:
!feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_len" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
Materializing 2 feature views to 2025-04-08 12:26:27+00:00 into the sqlite online store.

driver_hourly_stats from 2025-04-07 12:26:33+00:00 to 2025-04-08 12:26:27+00:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 375.34it/s]
driver_hourly_stats_fresh from 2025-04-07 12:26:33+00:00 to 2025-04-08 12:26:27+00:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 581.09it/s]


This tells Feast:
➤ “Take all new feature data since the last materialization point and load it into the online store, up to the current UTC timestamp.”

It ensures that your online feature store is up to date, especially for features computed periodically (e.g., hourly/daily aggregates).

In [24]:
# Retrieve latest features for a driver
online_features = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips"
    ],
    entity_rows=[{"driver_id": int(entity_df.iloc[0]['driver_id'])}]
).to_dict()

# Prepare for prediction
online_X = pd.DataFrame.from_dict(online_features).drop(columns=["driver_id"])
expected_columns = ["conv_rate", "acc_rate", "avg_daily_trips"]
online_X = online_X[expected_columns]

# Predict
real_time_prediction = model.predict(online_X)

print("Real-time feature vector:\n", online_X)
print("Predicted High Performance?", real_time_prediction[0])


Real-time feature vector:
    conv_rate  acc_rate  avg_daily_trips
0   0.974777  0.451735              500
Predicted High Performance? 1


With Feast:
You registered features once using feast apply.

You retrieved training data with automatic point-in-time joins using get_historical_features().

You served features in real-time with low latency using get_online_features().

You didn’t have to build separate pipelines for training vs. inference – Feast took care of that.

Using Feast introduces a standardized, scalable, and reliable way to manage ML features. It drastically reduces technical debt, operational risk, and duplicate work. It’s especially valuable in real-world production ML systems where data freshness, feature consistency, and collaboration are critical.

This is why Feast is often considered an essential part of the modern MLOps stack.